# Summary
This notebook builds the candidate universe of $p$-persistent arcs, for both single-scale and multiscale causal backbones. 

# Import libraries

In [1]:
import itertools
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import sys

sys.path.insert(0, r"..\code")

from copy import deepcopy
from utils import save_obj,load_obj


In [2]:
data_dir="../data/"
ts_dir="../data/TimeSeriesAAL/" 
processed="../data/processed/"
diffreg="../data/processed/diff_regions/"

replace=False

# Names and index

In [3]:
#names
names = load_obj('region_names', processed)
idx_left = load_obj('index_left_regions',processed)
idx_right = load_obj('index_right_regions',processed)

names_left_df=names.iloc[idx_left].copy()
names_right_df=names.iloc[idx_right].copy()

names_left=names_left_df['Region']
names_right=names_right_df['Region']

names_reidx=names_left_df.copy()

names_reidx.reset_index(inplace=True, drop=True)
names_reidx.reset_index(inplace=True, drop=False)
names_reidx.set_index('Region', drop=True, inplace=True)

def retrieve_index(x, what):
    return names_reidx.loc[x[what]]['index']

# Persistent edges SS

In [4]:
if replace: writer = pd.ExcelWriter('1.1_single_scale_edges.xlsx')

threshold=80
alow, aup = (5, 95)
hemis=['right', 'left']
exams=[0, 1]
edges_df=pd.DataFrame()

for hemi,exam in itertools.product(hemis,exams):
    C_spk = load_obj('1.0_ss_{}_hemi_exam{}'.format(hemi,exam), processed).transpose((2,0,1))

    #occurrence
    C_occur=np.sign(np.abs(C_spk)).sum(axis=0)
    C_occur_df=pd.DataFrame(C_occur, index=names_right, columns=names_right)
    G=nx.from_pandas_adjacency(C_occur_df, create_using=nx.DiGraph)
    to_exclude=[(a,b) for a, b, attrs in G.edges(data=True) if attrs["weight"]<=threshold]
    [G.remove_edge(a,b) for a,b in to_exclude]
    G.remove_nodes_from(list(nx.isolates(G)))

    #percentiles
    C_percentile_alow=np.percentile(C_spk, alow, axis=0)
    C_percentile_aup=np.percentile(C_spk, aup, axis=0)

    C_percentile_alow_df=pd.DataFrame(C_percentile_alow, index=names_right, columns=names_right)
    C_percentile_aup_df=pd.DataFrame(C_percentile_aup, index=names_right, columns=names_right)

    [nx.set_edge_attributes(G, {(a,b):{'{}%'.format(alow):C_percentile_alow_df.loc[a,b], '{}%'.format(aup):C_percentile_aup_df.loc[a,b]}}) for a,b in G.edges] 

    if edges_df.empty:
        edges_df=nx.to_pandas_edgelist(G)
        edges_df['hemi']=hemi
        edges_df['day']=exam
    else:
        item=nx.to_pandas_edgelist(G)
        item['hemi']=hemi
        item['day']=exam
        edges_df=pd.concat((edges_df, item))

edges_df.rename(columns={'weight':'occurrence (out of 100)'}, inplace=True)

edges_df['count']=1.
edges_df_count = (edges_df[['source', 'target', 'count']].groupby(['source', 'target']).sum()).copy()
edges_df_grouped = (edges_df.groupby(['source', 'target', 'occurrence (out of 100)', '5%', '95%', 'hemi', 'day']).sum()).copy()
display(edges_df_count)
display(edges_df_grouped)

edges_df['source_idx']=edges_df.apply(lambda x: retrieve_index(x, 'source'), axis=1)
edges_df['target_idx']=edges_df.apply(lambda x: retrieve_index(x, 'target'), axis=1)

if replace:
    edges_df.to_excel(writer, sheet_name='Raw', index=True)
    edges_df_count.to_excel(writer, sheet_name='Count', index=True)
    edges_df_grouped.to_excel(writer, sheet_name='Grouped', index=True)

    writer.save()
    writer.close()

count
source      target               
Angular     Precuneus         2.0
Calcarine   Lingual           2.0
Cereb I II  Amygdala          1.0
            Cereb III - VI    4.0
            Cereb VII - X     4.0
...                           ...
TempMid     TempSup           2.0
TempPole    ParaHippoc        2.0
TempSup     Amygdala          1.0
            Heschl            4.0
            RolandOperc       4.0

[74 rows x 1 columns]

count
source      target      occurrence (out of 100) 5%        95%      hemi  day       
Angular     Precuneus   87.0                     0.000000 0.462918 right 1      1.0
                        91.0                     0.000000 0.475662 right 0      1.0
Calcarine   Lingual     86.0                     0.000000 0.424295 left  0      1.0
                        92.0                     0.000000 0.431181 left  1      1.0
Cereb I II  Amygdala    81.0                    -0.348396 0.426262 right 0      1.0
...                                                                             ...
TempSup     Heschl      100.0                    0.235486 0.761342 right 1      1.0
            RolandOperc 88.0                     0.000000 0.406729 right 0      1.0
                        91.0                     0.000000 0.414250 right 1      1.0
                        97.0                     0.132678 0.447455 left  0      1.0
                                                 0.142039 0.413900 left  1      1.0

[193 rows x 1 columns]

# Persistent edges MS

In [5]:
if replace: writer = pd.ExcelWriter('1.1_multi_scale_edges.xlsx')

J=5
N=len(names_right)
threshold=80
alow, aup = (5, 95)
hemis=['right', 'left']
exams=[0, 1]

for scale in range(J):
    edges_df=pd.DataFrame()
    for hemi,exam in itertools.product(hemis,exams):
        C_spk = load_obj('1.0_ms_{}_hemi_exam{}'.format(hemi,exam), processed).transpose((2,0,1))[:,scale*N:(scale+1)*N,scale*N:(scale+1)*N]

        #occurrence
        C_occur=np.sign(np.abs(C_spk)).sum(axis=0)
        C_occur_df=pd.DataFrame(C_occur, index=names_right, columns=names_right)
        G=nx.from_pandas_adjacency(C_occur_df, create_using=nx.DiGraph)
        to_exclude=[(a,b) for a, b, attrs in G.edges(data=True) if attrs["weight"]<=threshold]
        [G.remove_edge(a,b) for a,b in to_exclude]
        G.remove_nodes_from(list(nx.isolates(G)))

        #percentiles
        C_percentile_alow=np.percentile(C_spk, alow, axis=0)
        C_percentile_aup=np.percentile(C_spk, aup, axis=0)

        C_percentile_alow_df=pd.DataFrame(C_percentile_alow, index=names_right, columns=names_right)
        C_percentile_aup_df=pd.DataFrame(C_percentile_aup, index=names_right, columns=names_right)

        [nx.set_edge_attributes(G, {(a,b):{'{}%'.format(alow):C_percentile_alow_df.loc[a,b], '{}%'.format(aup):C_percentile_aup_df.loc[a,b]}}) for a,b in G.edges] 


        if edges_df.empty:
            edges_df=nx.to_pandas_edgelist(G)
            edges_df['hemi']=hemi
            edges_df['day']=exam
            edges_df['scale']=J-scale
        else:
            item=nx.to_pandas_edgelist(G)
            item['hemi']=hemi
            item['day']=exam
            item['scale']=J-scale
            edges_df=pd.concat((edges_df, item))

    edges_df.rename(columns={'weight':'occurrence (out of 100)'}, inplace=True)

    edges_df['count']=1.
    edges_df_count = (edges_df[['source', 'target', 'count']].groupby(['source', 'target']).sum()).copy()
    edges_df_grouped = (edges_df.groupby(['source', 'target', 'occurrence (out of 100)', '5%', '95%', 'hemi', 'day', 'scale']).sum()).copy()
    display(edges_df_count)
    display(edges_df_grouped)

    if replace:
        edges_df['source_idx']=edges_df.apply(lambda x: retrieve_index(x, 'source'), axis=1)
        edges_df['target_idx']=edges_df.apply(lambda x: retrieve_index(x, 'target'), axis=1)
        edges_df.to_excel(writer, sheet_name='Raw_scale{}'.format(J-scale), index=True)
        edges_df_count.to_excel(writer, sheet_name='Count_scale{}'.format(J-scale), index=True)
        edges_df_grouped.to_excel(writer, sheet_name='Grouped_scale{}'.format(J-scale), index=True)

if replace: 
    writer.save()
    writer.close()

count
source      target          
Angular     Amygdala     2.0
            Olfactory    1.0
Calcarine   Amygdala     1.0
Cereb I II  Amygdala     1.0
            Caudate      1.0
...                      ...
TempSup     Heschl       4.0
            Olfactory    1.0
Thalamus    Amygdala     1.0
Vermis      Amygdala     3.0
            Olfactory    2.0

[131 rows x 1 columns]

count
source      target    occurrence (out of 100) 5%        95%      hemi  day scale       
Angular     Amygdala  82.0                    -1.142782 1.007516 right 1   5        1.0
                      88.0                    -0.859750 0.906163 left  1   5        1.0
            Olfactory 87.0                    -1.394124 1.090230 right 1   5        1.0
Calcarine   Amygdala  85.0                    -0.683278 0.697444 left  0   5        1.0
Cereb I II  Amygdala  82.0                    -1.151250 1.148205 right 1   5        1.0
...                                                                                 ...
Vermis      Amygdala  82.0                    -1.027009 1.031824 left  0   5        1.0
                      84.0                    -1.011451 0.957894 right 1   5        1.0
                      89.0                    -1.002237 0.990069 left  1   5        1.0
            Olfactory 82.0                    -0.999030 0.950797 right 1   5        1.0
                      84.0                    -0.706467 1.017759 left  1   5        1.0

[266 rows x 1 columns]

count
source      target           
Angular     Amygdala      2.0
            Olfactory     1.0
Cereb I II  Amygdala      2.0
            Caudate       1.0
            Olfactory     2.0
...                       ...
TempPole    Olfactory     3.0
            ParaHippoc    2.0
TempSup     Amygdala      2.0
            Heschl        4.0
            Olfactory     1.0

[130 rows x 1 columns]

count
source      target    occurrence (out of 100) 5%        95%      hemi  day scale       
Angular     Amygdala  81.0                    -0.861443 0.812043 right 1   4        1.0
                      85.0                    -0.817282 0.989372 right 0   4        1.0
            Olfactory 90.0                    -1.082880 0.948293 right 0   4        1.0
Cereb I II  Amygdala  82.0                    -0.711986 0.913327 right 0   4        1.0
                                              -0.534417 0.847324 right 1   4        1.0
...                                                                                 ...
TempSup     Heschl    96.0                     0.148601 1.387016 right 0   4        1.0
                                               0.190602 1.352707 left  1   4        1.0
                      97.0                     0.216182 1.383059 left  0   4        1.0
                      98.0                     0.105799 1.407117 right 1   4        1.0
            Olfactory 81.0                    -0.714645 0.925978 right 0   4        1.0

[273 rows x 1 columns]

count
source      target               
Angular     Amygdala          1.0
            Olfactory         1.0
Cereb I II  Amygdala          1.0
            Caudate           2.0
            Cereb III - VI    4.0
...                           ...
TempPole    ParaHippoc        1.0
TempSup     Amygdala          1.0
            Heschl            4.0
            Olfactory         2.0
            RolandOperc       1.0

[103 rows x 1 columns]

count
source      target      occurrence (out of 100) 5%        95%      hemi  day scale       
Angular     Amygdala    84.0                    -0.856570 0.580818 left  1   3        1.0
            Olfactory   81.0                    -1.115781 0.765165 right 0   3        1.0
Cereb I II  Amygdala    83.0                    -0.682687 0.940170 right 1   3        1.0
            Caudate     85.0                    -0.177957 0.826119 right 1   3        1.0
                        88.0                    -0.141794 0.899530 right 0   3        1.0
...                                                                                   ...
TempSup     Heschl      99.0                     0.261072 1.164875 left  0   3        1.0
                                                 0.274824 1.216788 right 0   3        1.0
            Olfactory   84.0                    -0.858103 0.875349 right 0   3        1.0
                        88.0                    -0.823820 0.812344 right 1   3        1.0
            RolandOperc 92.0                     0.000000 0.585741 left  1   3        1.0

[238 rows x 1 columns]

count
source      target               
Angular     Amygdala          4.0
            Olfactory         3.0
Calcarine   Lingual           1.0
Cereb I II  Amygdala          3.0
            Cereb III - VI    1.0
...                           ...
TempPole    ParaHippoc        1.0
TempSup     Amygdala          1.0
            Heschl            4.0
            Olfactory         2.0
            RolandOperc       3.0

[88 rows x 1 columns]

count
source  target      occurrence (out of 100) 5%        95%      hemi  day scale       
Angular Amygdala    82.0                    -0.890777 1.020892 right 0   2        1.0
                    83.0                    -0.828402 0.437834 left  1   2        1.0
                    86.0                    -0.886592 0.959228 right 1   2        1.0
                                            -0.667223 0.711911 left  0   2        1.0
        Olfactory   81.0                    -1.116322 0.777662 right 0   2        1.0
...                                                                               ...
TempSup Olfactory   83.0                    -0.946348 0.900253 right 1   2        1.0
                    88.0                    -0.920016 1.276154 right 0   2        1.0
        RolandOperc 81.0                     0.000000 0.525283 right 1   2        1.0
                    85.0                     0.000000 0.477032 left  0   2        1.0
                    91.0                     0.000000 0.484215 left  1   2        1.0

[200 rows x 1 columns]

count
source      target               
Angular     Amygdala          2.0
            Olfactory         3.0
Cereb I II  Amygdala          1.0
            Cereb III - VI    1.0
            Olfactory         1.0
...                           ...
TempMid     Olfactory         3.0
            TempInf           2.0
            TempPole          1.0
            TempSup           2.0
TempSup     Olfactory         1.0

[64 rows x 1 columns]

count
source  target    occurrence (out of 100) 5%        95%      hemi  day scale       
Angular Amygdala  84.0                    -0.709918 0.818070 right 0   1        1.0
                  90.0                    -0.629205 0.662256 left  1   1        1.0
        Olfactory 83.0                    -0.504813 0.607947 left  1   1        1.0
                  84.0                    -0.917469 0.945559 right 1   1        1.0
                                          -0.901023 1.158624 right 0   1        1.0
...                                                                             ...
TempMid TempInf   85.0                     0.000000 0.465155 right 1   1        1.0
        TempPole  86.0                     0.000000 0.462630 right 1   1        1.0
        TempSup   83.0                     0.000000 0.422007 left  1   1        1.0
                  93.0                     0.000000 0.529978 left  0   1        1.0
TempSup Olfactory 85.0                    -0.984523 0.757360 right 1   1        1.0

[142 rows x 1 columns]